# Setting some Thresholds for Rainfall

In [1]:
%load_ext jupyter_black
import os
import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd
import warnings
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta

In [2]:
warnings.filterwarnings("ignore")
AA_DATA_DIR = Path(os.getenv("AA_DATA_DIR"))
AA_DATA_DIR_NEW = Path(os.getenv("AA_DATA_DIR_NEW"))

ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]

In [3]:
emdat_path = (
    AA_DATA_DIR_NEW
    / "private"
    / "processed"
    / "glb"
    / "emdat"
    / "emdat-tropicalcyclone-2000-2022-processed-sids.csv"
)

ibtracs_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "glb"
    / "ibtracs"
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)

adm1_path = (
    AA_DATA_DIR
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)

df_emdat = pd.read_csv(emdat_path)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_ibtracs = gpd.read_file(ibtracs_path)

gdf_adm1_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]
df_emdat = df_emdat[df_emdat.iso3 == "MOZ"]  # Just want the impacts to Mozambique

In [5]:
imerg_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "processed"
    / "moz"
    / "daily_imerg_cyclone_landfall.csv"
)

df_imerg = pd.read_csv(imerg_path)
df_imerg["date"] = pd.to_datetime(df_imerg["date"])

## Get the rainfall per day

In [11]:
# Calculate the mean of median_precip_250km for each time_step
mean_precip = df_imerg.groupby("time_step")["median_precip_250km"].mean().reset_index()

In [13]:
# Plot with Plotly
fig = px.line(
    mean_precip,
    x="time_step",
    y="median_precip_250km",
    title="Mean Median Precipitation (250km) by Time Step",
    labels={
        "time_step": "Days to Cyclone Landfall",
        "median_precip_250km": "Mean of Median Precipitation in 250km radius",
    },
)
fig.update_traces(mode="markers+lines")
fig.show()

## Rainfall by Storm

In [24]:
# Filter data for time_step = 0
df_time_step_0 = df_imerg[df_imerg["time_step"] == 0]

# Sort the values by median_precip_250km in descending order
df_time_step_0_sorted = df_time_step_0.sort_values(
    by="median_precip_250km", ascending=False
)

# Plot with Plotly
fig = px.bar(
    df_time_step_0_sorted,
    x="median_precip_250km",
    y="storm",
    title="Rainfall on Landfall Date for Each Storm",
    labels={
        "storm": "Storm",
        "median_precip_250km": "Median Precipitation (in 250km radius)",
    },
    orientation="h",
)
fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=600)
fig.show()

## Setting threshold to 40mm on landfall date

In [27]:
df_time_step_0[df_time_step_0["median_precip_250km"] >= 40]

,Unnamed: 0,storm,date,time_step,median_precip_250km
2,2,FAVIO,2007-02-22 12:00:00,0,40.150627
138,138,ANA,2022-01-24 09:00:00,0,63.624435
146,146,GOMBE,2022-03-11 03:00:00,0,79.764084
162,162,FREDDY,2023-03-11 18:00:00,0,46.452620
